# 🎤 Whisper-Base Sinhala ASR - Sample Training (100 Records)

This notebook trains Whisper-base model on a small sample of Sinhala ASR data for testing and demonstration purposes.

## 📋 Workflow:
1. Load clean train/test datasets
2. Combine datasets
3. Sample 100 records randomly
4. Split 80/20 (80 train, 20 test)
5. Train Whisper-base model
6. Evaluate performance

## 🎯 Dataset Info:
- **Sample Size**: 100 records
- **Training**: 80 samples
- **Testing**: 20 samples
- **Language**: Sinhala (සිංහල)
- **Model**: whisper-base

## 1. Environment Setup

In [1]:
# Install required packages
!pip install transformers datasets torch torchaudio librosa soundfile evaluate jiwer accelerate tensorboard --quiet

In [2]:
# Import libraries
import os
import pandas as pd
import numpy as np
import torch
import librosa
from pathlib import Path
import json
import warnings
warnings.filterwarnings('ignore')

from transformers import (
    WhisperFeatureExtractor,
    WhisperTokenizer,
    WhisperProcessor,
    WhisperForConditionalGeneration,
    TrainingArguments,
    Trainer,
    EarlyStoppingCallback
)
from datasets import Dataset, Audio
import evaluate
from sklearn.model_selection import train_test_split

# Set seeds
import random
random.seed(42)
np.random.seed(42)
torch.manual_seed(42)
if torch.cuda.is_available():
    torch.cuda.manual_seed_all(42)

print("🔧 Environment Setup Complete!")
print(f"📱 Device: {'CUDA' if torch.cuda.is_available() else 'CPU'}")
print(f"🧠 GPU: {torch.cuda.get_device_name(0) if torch.cuda.is_available() else 'None'}")
if torch.cuda.is_available():
    print(f"💾 CUDA Memory: {torch.cuda.get_device_properties(0).total_memory // 1e9:.1f}GB")


🔧 Environment Setup Complete!
📱 Device: CPU
🧠 GPU: None
🔧 Environment Setup Complete!
📱 Device: CPU
🧠 GPU: None


## 2. Load and Combine Clean Datasets

In [3]:
# Define paths
data_dir = Path(r"f:\UOK Fourth Year\Research\my research\dataset\sinhala asr")
processed_dir = data_dir / "processed_asr_data"
sample_output_dir = data_dir / "sample_models"
sample_output_dir.mkdir(exist_ok=True)

print("📂 Loading clean preprocessed datasets...")

# Load clean train and test data
train_clean = pd.read_csv(processed_dir / "train_data_clean.csv")
test_clean = pd.read_csv(processed_dir / "test_data_clean.csv")

print(f"✅ Clean datasets loaded:")
print(f"   📊 Clean training data: {len(train_clean):,} samples")
print(f"   📊 Clean test data: {len(test_clean):,} samples")

# Combine all clean data
combined_clean = pd.concat([train_clean, test_clean], ignore_index=True)
print(f"   📊 Combined clean data: {len(combined_clean):,} samples")

# Show sample
print(f"\n📋 Sample combined data:")
print(combined_clean.head(3))
print(f"\n📋 Columns: {combined_clean.columns.tolist()}")

📂 Loading clean preprocessed datasets...
✅ Clean datasets loaded:
   📊 Clean training data: 72,155 samples
   📊 Clean test data: 18,039 samples
   📊 Combined clean data: 90,194 samples

📋 Sample combined data:
                                  file  \
0  asr_sinhala/data/98/983e3c6613.flac   
1  asr_sinhala/data/29/29ab15c6d4.flac   
2  asr_sinhala/data/b0/b0072f9ac0.flac   

                                sentence_cleaned  
0   එය මිලිටරිය තුළ ති‍යන ප්‍රධානම ප්‍රතිමානයක්.  
1  සාහිත්‍යකරුවාට ඊට වැඩිය ලොකු වගකීමක් තියෙනවා.  
2                        ඕගොල්ලන්ට දකින්න ලැබෙයි  

📋 Columns: ['file', 'sentence_cleaned']
✅ Clean datasets loaded:
   📊 Clean training data: 72,155 samples
   📊 Clean test data: 18,039 samples
   📊 Combined clean data: 90,194 samples

📋 Sample combined data:
                                  file  \
0  asr_sinhala/data/98/983e3c6613.flac   
1  asr_sinhala/data/29/29ab15c6d4.flac   
2  asr_sinhala/data/b0/b0072f9ac0.flac   

                                senten

## 3. Sample 100 Records and Split 80/20

In [4]:
# Sample 100 records randomly
SAMPLE_SIZE = 10
print(f"🎲 Sampling {SAMPLE_SIZE} records from {len(combined_clean):,} total records...")

# Ensure we have enough data
if len(combined_clean) < SAMPLE_SIZE:
    SAMPLE_SIZE = len(combined_clean)
    print(f"⚠️ Adjusted sample size to {SAMPLE_SIZE} (all available data)")

# Random sampling
sample_data = combined_clean.sample(n=SAMPLE_SIZE, random_state=42).reset_index(drop=True)

print(f"✅ Sampled {len(sample_data)} records")

# Split 80/20 for train/test
train_sample, test_sample = train_test_split(
    sample_data, 
    test_size=0.2, 
    random_state=42, 
    shuffle=True
)

print(f"\n📊 Sample data split:")
print(f"   🚀 Training samples: {len(train_sample)}")
print(f"   🧪 Test samples: {len(test_sample)}")
print(f"   📈 Split ratio: {len(train_sample)/len(sample_data)*100:.0f}% / {len(test_sample)/len(sample_data)*100:.0f}%")

# Reset indices
train_sample = train_sample.reset_index(drop=True)
test_sample = test_sample.reset_index(drop=True)

# Show samples
print(f"\n📝 Training sample:")
print(train_sample.head(2))
print(f"\n📝 Test sample:")
print(test_sample.head(2))

🎲 Sampling 10 records from 90,194 total records...
✅ Sampled 10 records

📊 Sample data split:
   🚀 Training samples: 8
   🧪 Test samples: 2
   📈 Split ratio: 80% / 20%

📝 Training sample:
                                  file   sentence_cleaned
0  asr_sinhala/data/5b/5b94883263.flac   දෙවනුව කඩේ යන්නේ
1  asr_sinhala/data/a2/a22f6ae045.flac  හැමෝම අනිවාර්යෙන්

📝 Test sample:
                                  file        sentence_cleaned
0  asr_sinhala/data/85/858c021de7.flac           ස්වභාවික හේතු
1  asr_sinhala/data/b9/b9666683a1.flac  ඇති හැකි ඇත්තෝ පෙරට ආහ


## 4. Validate Sample Audio Files

In [5]:
# Validate audio files in sample
def validate_sample_audio(df, data_dir, name="dataset"):
    """Validate all audio files in the sample"""
    print(f"🔍 Validating {name} audio files...")
    
    valid_files = []
    invalid_files = []
    durations = []
    
    for idx, row in df.iterrows():
        file_path = data_dir / row['file']
        
        try:
            if file_path.exists():
                # Load audio to validate
                audio, sr = librosa.load(file_path, sr=16000)
                duration = len(audio) / sr
                
                if 0.5 <= duration <= 30:  # Valid duration range
                    valid_files.append(idx)
                    durations.append(duration)
                else:
                    invalid_files.append((idx, f"Duration {duration:.2f}s out of range"))
            else:
                invalid_files.append((idx, "File not found"))
                
        except Exception as e:
            invalid_files.append((idx, f"Error: {e}"))
    
    print(f"   ✅ Valid files: {len(valid_files)}/{len(df)}")
    print(f"   ❌ Invalid files: {len(invalid_files)}")
    
    if durations:
        print(f"   🎵 Duration stats: {np.mean(durations):.2f}s avg, {np.min(durations):.2f}s min, {np.max(durations):.2f}s max")
    
    if invalid_files:
        print(f"   ⚠️ Invalid files:")
        for idx, reason in invalid_files[:3]:  # Show first 3
            print(f"      • Row {idx}: {reason}")
        if len(invalid_files) > 3:
            print(f"      • ... and {len(invalid_files) - 3} more")
    
    return valid_files, invalid_files

# Validate both train and test samples
train_valid, train_invalid = validate_sample_audio(train_sample, data_dir, "training")
test_valid, test_invalid = validate_sample_audio(test_sample, data_dir, "test")

# Filter out invalid files
if train_invalid:
    invalid_indices = [idx for idx, _ in train_invalid]
    train_sample = train_sample.drop(invalid_indices).reset_index(drop=True)
    print(f"🧹 Removed {len(invalid_indices)} invalid training samples")

if test_invalid:
    invalid_indices = [idx for idx, _ in test_invalid]
    test_sample = test_sample.drop(invalid_indices).reset_index(drop=True)
    print(f"🧹 Removed {len(invalid_indices)} invalid test samples")

print(f"\n📊 Final sample sizes:")
print(f"   🚀 Training: {len(train_sample)} samples")
print(f"   🧪 Testing: {len(test_sample)} samples")
print(f"   📊 Total: {len(train_sample) + len(test_sample)} samples")

🔍 Validating training audio files...
   ✅ Valid files: 8/8
   ❌ Invalid files: 0
   🎵 Duration stats: 3.59s avg, 2.30s min, 4.90s max
🔍 Validating test audio files...
   ✅ Valid files: 2/2
   ❌ Invalid files: 0
   🎵 Duration stats: 3.30s avg, 2.60s min, 4.00s max

📊 Final sample sizes:
   🚀 Training: 8 samples
   🧪 Testing: 2 samples
   📊 Total: 10 samples
   ✅ Valid files: 8/8
   ❌ Invalid files: 0
   🎵 Duration stats: 3.59s avg, 2.30s min, 4.90s max
🔍 Validating test audio files...
   ✅ Valid files: 2/2
   ❌ Invalid files: 0
   🎵 Duration stats: 3.30s avg, 2.60s min, 4.00s max

📊 Final sample sizes:
   🚀 Training: 8 samples
   🧪 Testing: 2 samples
   📊 Total: 10 samples


## 5. Initialize Whisper Components

In [6]:
# Initialize Whisper model and components
model_name = "openai/whisper-base"
print(f"🤖 Loading Whisper components: {model_name}")

# Load components
feature_extractor = WhisperFeatureExtractor.from_pretrained(model_name)
tokenizer = WhisperTokenizer.from_pretrained(model_name, language="si", task="transcribe")
processor = WhisperProcessor.from_pretrained(model_name, language="si", task="transcribe")
model = WhisperForConditionalGeneration.from_pretrained(model_name)

# Configure for Sinhala
model.generation_config.language = "si"
model.generation_config.task = "transcribe"
model.generation_config.forced_decoder_ids = None

# Enable optimizations
model.gradient_checkpointing_enable()

print(f"✅ Whisper loaded successfully:")
print(f"   🧠 Parameters: {model.num_parameters():,}")
print(f"   🎯 Language: {model.generation_config.language}")
print(f"   🎯 Task: {model.generation_config.task}")
print(f"   🎵 Sampling rate: {feature_extractor.sampling_rate}Hz")
print(f"   💾 Gradient checkpointing: Enabled")

🤖 Loading Whisper components: openai/whisper-base
✅ Whisper loaded successfully:
   🧠 Parameters: 72,593,920
   🎯 Language: si
   🎯 Task: transcribe
   🎵 Sampling rate: 16000Hz
   💾 Gradient checkpointing: Enabled
✅ Whisper loaded successfully:
   🧠 Parameters: 72,593,920
   🎯 Language: si
   🎯 Task: transcribe
   🎵 Sampling rate: 16000Hz
   💾 Gradient checkpointing: Enabled


## 6. Prepare Training Data

In [7]:
# Convert sample data to HuggingFace Dataset format
def create_dataset(df, data_dir, name="dataset"):
    """Create HuggingFace Dataset from DataFrame"""
    print(f"🔄 Creating {name} dataset...")
    
    dataset_dict = {
        "audio": [],
        "text": []
    }
    
    for idx, row in df.iterrows():
        file_path = data_dir / row['file']
        
        try:
            # Load audio
            audio, sr = librosa.load(file_path, sr=16000)
            
            # Store as dict format that HuggingFace expects
            dataset_dict["audio"].append({
                "array": audio.astype(np.float32),
                "sampling_rate": 16000
            })
            dataset_dict["text"].append(row['sentence_cleaned'])
            
        except Exception as e:
            print(f"   ⚠️ Skipping {file_path}: {e}")
    
    print(f"   ✅ Created {len(dataset_dict['audio'])} samples")
    
    # Create HuggingFace Dataset
    dataset = Dataset.from_dict(dataset_dict)
    
    return dataset

# Create datasets
train_dataset = create_dataset(train_sample, data_dir, "training")
test_dataset = create_dataset(test_sample, data_dir, "test")

print(f"\n📊 Datasets created:")
print(f"   🚀 Training dataset: {len(train_dataset)} samples")
print(f"   🧪 Test dataset: {len(test_dataset)} samples")

🔄 Creating training dataset...
   ✅ Created 8 samples
🔄 Creating test dataset...
   ✅ Created 2 samples

📊 Datasets created:
   🚀 Training dataset: 8 samples
   🧪 Test dataset: 2 samples


In [8]:
# Preprocessing function
def prepare_batch(batch):
    """Preprocess batch for Whisper training"""
    audio = batch["audio"]
    
    # Extract features
    batch["input_features"] = feature_extractor(
        audio["array"], 
        sampling_rate=audio["sampling_rate"]
    ).input_features[0]
    
    # Tokenize text
    batch["labels"] = tokenizer(batch["text"]).input_ids
    
    return batch

# Apply preprocessing
print("🔄 Preprocessing datasets...")
train_dataset = train_dataset.map(prepare_batch, remove_columns=train_dataset.column_names)
test_dataset = test_dataset.map(prepare_batch, remove_columns=test_dataset.column_names)

print(f"✅ Preprocessing complete!")
print(f"   📊 Input features shape: {np.array(train_dataset[0]['input_features']).shape}")
print(f"   📊 Labels length: {len(train_dataset[0]['labels'])}")
print(f"   📝 Sample text: {tokenizer.decode(train_dataset[0]['labels'][:30])}...")

🔄 Preprocessing datasets...


Map:   0%|          | 0/8 [00:00<?, ? examples/s]

Map:   0%|          | 0/2 [00:00<?, ? examples/s]

✅ Preprocessing complete!
   📊 Input features shape: (80, 3000)
   📊 Labels length: 33
   📝 Sample text: <|startoftranscript|><|si|><|transcribe|><|notimestamps|>දෙවනුව කඩේ යන්න...


## 7. Training Setup

In [ ]:
# Data collator
from dataclasses import dataclass
from typing import Any, Dict, List, Union

@dataclass
class DataCollatorSpeechSeq2SeqWithPadding:
    processor: Any

    def __call__(self, features: List[Dict[str, Union[List[int], torch.Tensor]]]) -> Dict[str, torch.Tensor]:
        input_features = [{"input_features": feature["input_features"]} for feature in features]
        batch = self.processor.feature_extractor.pad(input_features, return_tensors="pt")

        label_features = [{"input_ids": feature["labels"]} for feature in features]
        labels_batch = self.processor.tokenizer.pad(label_features, return_tensors="pt")

        labels = labels_batch["input_ids"].masked_fill(labels_batch.attention_mask.ne(1), -100)

        if (labels[:, 0] == self.processor.tokenizer.bos_token_id).all().cpu().item():
            labels = labels[:, 1:]

        batch["labels"] = labels
        return batch

data_collator = DataCollatorSpeechSeq2SeqWithPadding(processor=processor)

# Evaluation metric
wer_metric = evaluate.load("wer")

print("✅ Training components ready:")
print("   📦 Data collator: Configured")
print("   📊 Metrics: WER (Word Error Rate)")

✅ Training components ready:
   📦 Data collator: Configured
   📊 Metrics: WER (Word Error Rate)


In [43]:
# Improved compute_metrics function for training evaluation with multiple metrics
def compute_metrics_training(eval_pred):
    """Compute WER, CER, TER, and SER during training evaluations with proper error handling"""
    try:
        pred_ids = eval_pred.predictions
        label_ids = eval_pred.label_ids
        
        # Handle tuple predictions (from generation)
        if isinstance(pred_ids, tuple):
            pred_ids = pred_ids[0]
        
        # Convert to numpy arrays if needed
        if not isinstance(pred_ids, np.ndarray):
            pred_ids = np.array(pred_ids)
        if not isinstance(label_ids, np.ndarray):
            label_ids = np.array(label_ids)
        
        # Replace -100 with pad token for proper decoding
        label_ids = np.where(label_ids != -100, label_ids, tokenizer.pad_token_id)
        
        # Decode predictions and labels
        try:
            pred_str = tokenizer.batch_decode(pred_ids, skip_special_tokens=True)
            label_str = tokenizer.batch_decode(label_ids, skip_special_tokens=True)
        except Exception as decode_error:
            print(f"⚠️ Decode error: {decode_error}")
            # Return default high error rates for failed decoding
            return {
                "wer": 100.0,
                "cer": 100.0, 
                "ter": 100.0,
                "ser": 100.0
            }
        
        # Clean empty predictions/references
        valid_pairs = []
        for pred, ref in zip(pred_str, label_str):
            pred_clean = pred.strip() if pred else ""
            ref_clean = ref.strip() if ref else ""
            if pred_clean and ref_clean:  # Only include non-empty pairs
                valid_pairs.append((pred_clean, ref_clean))
        
        if not valid_pairs:
            print("⚠️ No valid prediction pairs found")
            return {
                "wer": 100.0,
                "cer": 100.0,
                "ter": 100.0, 
                "ser": 100.0
            }
        
        # Extract predictions and references
        clean_predictions = [pair[0] for pair in valid_pairs]
        clean_references = [pair[1] for pair in valid_pairs]
        
        # Compute metrics
        try:
            # WER (Word Error Rate)
            wer = wer_metric.compute(predictions=clean_predictions, references=clean_references) * 100
            
            # CER (Character Error Rate) - approximate using character-level comparison
            cer_total = 0
            for pred, ref in zip(clean_predictions, clean_references):
                # Simple character-level distance approximation
                pred_chars = list(pred.replace(' ', ''))
                ref_chars = list(ref.replace(' ', ''))
                if len(ref_chars) > 0:
                    char_errors = sum(p != r for p, r in zip(pred_chars, ref_chars))
                    char_errors += abs(len(pred_chars) - len(ref_chars))
                    cer_total += char_errors / len(ref_chars)
            cer = (cer_total / len(clean_predictions)) * 100 if clean_predictions else 100.0
            
            # TER (Translation Error Rate) - approximate as WER for simplicity
            ter = wer  # Using WER as TER approximation
            
            # SER (Sentence Error Rate) - percentage of completely incorrect sentences
            correct_sentences = sum(1 for pred, ref in zip(clean_predictions, clean_references) if pred.strip() == ref.strip())
            ser = ((len(clean_predictions) - correct_sentences) / len(clean_predictions)) * 100
            
        except Exception as metric_error:
            print(f"⚠️ Metric computation error: {metric_error}")
            return {
                "wer": 100.0,
                "cer": 100.0,
                "ter": 100.0,
                "ser": 100.0
            }
        
        print(f"📊 Evaluation Metrics (on {len(valid_pairs)} samples):")
        print(f"   WER: {wer:.1f}%, CER: {cer:.1f}%, TER: {ter:.1f}%, SER: {ser:.1f}%")
        
        return {
            "wer": wer,
            "cer": cer, 
            "ter": ter,
            "ser": ser
        }
        
    except Exception as e:
        print(f"⚠️ Error in compute_metrics: {e}")
        return {
            "wer": 100.0,
            "cer": 100.0,
            "ter": 100.0,
            "ser": 100.0
        }

print("✅ Improved compute_metrics function ready for training evaluation!")
print("   📊 Metrics: WER (%), CER (%), TER (%), SER (%)")

✅ Improved compute_metrics function ready for training evaluation!
   📊 Metrics: WER (%), CER (%), TER (%), SER (%)


In [38]:
# Training arguments - optimized for small dataset
training_args = TrainingArguments(
    output_dir=str(sample_output_dir / "whisper-base-sinhala-sample"),
    
    # Training settings for small dataset
    per_device_train_batch_size=4,
    per_device_eval_batch_size=2,
    gradient_accumulation_steps=2,  # Effective batch size = 4 * 2 = 8
    learning_rate=5e-5,  # Higher LR for faster convergence
    warmup_steps=20,     # More warmup for multi-epoch training
    num_train_epochs=5,  # Five epochs for better training
    weight_decay=0.01,
    
    # Optimization
    gradient_checkpointing=True,
    fp16=False,  # Disable FP16 for compatibility
    dataloader_num_workers=0,  # Avoid multiprocessing issues
    
    # Evaluation & Logging
    eval_strategy="steps",
    eval_steps=1,       # Evaluate every step to see WER progress
    save_steps=10,      # Save less frequently
    logging_steps=1,    # Log every step
    
    # Model saving
    save_total_limit=2,
    load_best_model_at_end=True,
    metric_for_best_model="wer",
    greater_is_better=False,
    
    # Reproducibility
    seed=42,
    
    # Progress tracking
    disable_tqdm=False,
    remove_unused_columns=True,
    
    # Disable wandb for simplicity
    report_to=[],
)

print("⚙️ Training configuration:")
print(f"   📊 Batch size: {training_args.per_device_train_batch_size} (effective: {training_args.per_device_train_batch_size * training_args.gradient_accumulation_steps})")
print(f"   📈 Learning rate: {training_args.learning_rate}")
print(f"   🏃 Epochs: {training_args.num_train_epochs}")
print(f"   🔥 Warmup steps: {training_args.warmup_steps}")
print(f"   💾 FP16: {training_args.fp16}")
print(f"   📂 Output: {training_args.output_dir}")

⚙️ Training configuration:
   📊 Batch size: 4 (effective: 8)
   📈 Learning rate: 5e-05
   🏃 Epochs: 5
   🔥 Warmup steps: 20
   💾 FP16: False
   📂 Output: f:\UOK Fourth Year\Research\my research\dataset\sinhala asr\sample_models\whisper-base-sinhala-sample


## 8. Initialize Trainer and Baseline Evaluation

In [44]:
# Initialize trainer
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=test_dataset,
    data_collator=data_collator,
    compute_metrics=compute_metrics_training,  # Enable WER computation during training
    tokenizer=processor.feature_extractor,
    callbacks=[EarlyStoppingCallback(early_stopping_patience=5)]
)

print("🚀 Trainer initialized!")
print(f"   📊 Training samples: {len(train_dataset)}")
print(f"   📊 Test samples: {len(test_dataset)}")
print(f"   🔄 Steps per epoch: {len(train_dataset) // (training_args.per_device_train_batch_size * training_args.gradient_accumulation_steps)}")

# Baseline evaluation
print("\n🧪 Running baseline evaluation...")
baseline_results = trainer.evaluate()

print(f"📊 Baseline results:")
for key, value in baseline_results.items():
    print(f"   • {key}: {value:.4f}")

baseline_loss = baseline_results['eval_loss']
print(f"\n🎯 Baseline Loss: {baseline_loss:.4f}")

🚀 Trainer initialized!
   📊 Training samples: 8
   📊 Test samples: 2
   🔄 Steps per epoch: 1

🧪 Running baseline evaluation...


⚠️ Decode error: int() argument must be a string, a bytes-like object or a real number, not 'list'
📊 Baseline results:
   • eval_loss: 2.5942
   • eval_model_preparation_time: 0.0060
   • eval_wer: 100.0000
   • eval_cer: 100.0000
   • eval_ter: 100.0000
   • eval_ser: 100.0000
   • eval_runtime: 2.4211
   • eval_samples_per_second: 0.8260
   • eval_steps_per_second: 0.4130

🎯 Baseline Loss: 2.5942


## 9. Model Training

In [45]:
# Start training
print("🚀 Starting Whisper fine-tuning on sample data...")
print(f"📊 Training on {len(train_dataset)} samples for {training_args.num_train_epochs} epochs")
print(f"⏰ Estimated time: ~{len(train_dataset) * training_args.num_train_epochs / 60:.1f} minutes")
print("\n" + "="*50)
print("         SAMPLE TRAINING STARTED")
print("="*50)

# Train the model
train_result = trainer.train()

print("\n" + "="*50)
print("        SAMPLE TRAINING COMPLETED")
print("="*50)

# Training summary
print(f"\n📊 Training Summary:")
print(f"   ⏱️ Time: {train_result.metrics['train_runtime']:.2f}s ({train_result.metrics['train_runtime']/60:.1f}m)")
print(f"   📉 Final loss: {train_result.metrics['train_loss']:.4f}")
print(f"   🚀 Samples/sec: {train_result.metrics['train_samples_per_second']:.2f}")

🚀 Starting Whisper fine-tuning on sample data...
📊 Training on 8 samples for 5 epochs
⏰ Estimated time: ~0.7 minutes

         SAMPLE TRAINING STARTED


Step,Training Loss,Validation Loss,Model Preparation Time,Wer,Cer,Ter,Ser
1,2.457700,2.594180,0.006000,100.000000,100.000000,100.000000,100.000000
2,2.464200,2.520189,0.006000,100.000000,100.000000,100.000000,100.000000
3,2.340300,2.373372,0.006000,100.000000,100.000000,100.000000,100.000000
4,2.130600,2.211012,0.006000,100.000000,100.000000,100.000000,100.000000
5,1.918000,2.110575,0.006000,100.000000,100.000000,100.000000,100.000000


⚠️ Decode error: int() argument must be a string, a bytes-like object or a real number, not 'list'
⚠️ Decode error: int() argument must be a string, a bytes-like object or a real number, not 'list'
⚠️ Decode error: int() argument must be a string, a bytes-like object or a real number, not 'list'
⚠️ Decode error: int() argument must be a string, a bytes-like object or a real number, not 'list'
⚠️ Decode error: int() argument must be a string, a bytes-like object or a real number, not 'list'
⚠️ Decode error: int() argument must be a string, a bytes-like object or a real number, not 'list'
⚠️ Decode error: int() argument must be a string, a bytes-like object or a real number, not 'list'
⚠️ Decode error: int() argument must be a string, a bytes-like object or a real number, not 'list'
⚠️ Decode error: int() argument must be a string, a bytes-like object or a real number, not 'list'


There were missing keys in the checkpoint model loaded: ['proj_out.weight'].



        SAMPLE TRAINING COMPLETED

📊 Training Summary:
   ⏱️ Time: 166.12s (2.8m)
   📉 Final loss: 2.2622
   🚀 Samples/sec: 0.24


## 10. Final Evaluation

In [36]:
# Final evaluation
print("🧪 Final model evaluation...")
final_results = trainer.evaluate()

print(f"\n📊 Final evaluation results:")
for key, value in final_results.items():
    print(f"   • {key}: {value:.4f}")

final_loss = final_results['eval_loss']
improvement = baseline_loss - final_loss

print(f"\n🎯 Results Comparison:")
print(f"   📉 Baseline Loss: {baseline_loss:.4f}")
print(f"   📈 Final Loss: {final_loss:.4f}")
print(f"   {'🎉' if improvement > 0 else '📊'} Change: {improvement:+.4f}")
if baseline_loss > 0:
    print(f"   📊 Relative change: {(improvement/baseline_loss)*100:+.1f}%")

🧪 Final model evaluation...


⚠️ Decode error: int() argument must be a string, a bytes-like object or a real number, not 'list'

📊 Final evaluation results:
   • eval_loss: 2.8509
   • eval_model_preparation_time: 0.0060
   • eval_wer: 100.0000
   • eval_runtime: 2.5691
   • eval_samples_per_second: 0.7780
   • eval_steps_per_second: 0.3890
   • epoch: 3.0000

🎯 Results Comparison:
   📉 Baseline Loss: 2.6045
   📈 Final Loss: 2.8509
   📊 Change: -0.2463
   📊 Relative change: -9.5%


## 11. Sample Predictions

In [37]:
# Test with sample predictions
print("🎤 Testing trained model with sample predictions...")

def predict_sample(dataset, index):
    """Make prediction on a sample"""
    sample = dataset[index]
    input_features = torch.tensor(sample['input_features']).unsqueeze(0)
    
    with torch.no_grad():
        predicted_ids = model.generate(input_features, max_length=225)
        
    prediction = tokenizer.decode(predicted_ids[0], skip_special_tokens=True)
    reference = tokenizer.decode(sample['labels'], skip_special_tokens=True)
    
    return prediction, reference

# Test on all test samples
print(f"\n📝 Sample Predictions on {len(test_dataset)} test cases:")
print("="*70)

all_predictions = []
all_references = []

for i in range(len(test_dataset)):
    prediction, reference = predict_sample(test_dataset, i)
    all_predictions.append(prediction)
    all_references.append(reference)
    
    if i < 5:  # Show first 5 samples in detail
        print(f"\n🎵 Test Sample {i+1}:")
        print(f"   📝 Reference:  {reference}")
        print(f"   🤖 Prediction: {prediction}")
        
        # Individual WER
        sample_wer = wer_metric.compute(predictions=[prediction], references=[reference])
        print(f"   📊 WER: {sample_wer*100:.1f}%")

# Overall WER on all predictions
overall_wer = wer_metric.compute(predictions=all_predictions, references=all_references)
print(f"\n🎯 Overall Test WER: {overall_wer*100:.2f}%")
print("="*70)

🎤 Testing trained model with sample predictions...

📝 Sample Predictions on 2 test cases:

🎵 Test Sample 1:
   📝 Reference:  ස්වභාවික හේතු
   🤖 Prediction:  Saab abi ki heitu.

🎵 Test Sample 1:
   📝 Reference:  ස්වභාවික හේතු
   🤖 Prediction:  Saab abi ki heitu.
   📊 WER: 200.0%
   📊 WER: 200.0%

🎵 Test Sample 2:
   📝 Reference:  ඇති හැකි ඇත්තෝ පෙරට ආහ
   🤖 Prediction:  Ati haki at do perita ha.
   📊 WER: 120.0%

🎯 Overall Test WER: 142.86%

🎵 Test Sample 2:
   📝 Reference:  ඇති හැකි ඇත්තෝ පෙරට ආහ
   🤖 Prediction:  Ati haki at do perita ha.
   📊 WER: 120.0%

🎯 Overall Test WER: 142.86%


## 12. Save Sample Model

In [29]:
# Save the trained sample model
print("💾 Saving sample model...")

sample_model_dir = sample_output_dir / "whisper-base-sinhala-sample-final"
sample_model_dir.mkdir(exist_ok=True)

# Save model and processor
trainer.save_model(str(sample_model_dir))
processor.save_pretrained(str(sample_model_dir))

# Save training metadata
metadata = {
    "model_type": "whisper-base-sinhala-sample",
    "base_model": model_name,
    "language": "sinhala",
    "sample_size": len(train_dataset) + len(test_dataset),
    "training_samples": len(train_dataset),
    "test_samples": len(test_dataset),
    "baseline_loss": baseline_loss,
    "final_loss": final_loss,
    "improvement": improvement,
    "training_time_seconds": train_result.metrics['train_runtime'],
    "final_train_loss": train_result.metrics['train_loss'],
    "overall_test_wer": overall_wer * 100,
    "training_config": {
        "epochs": training_args.num_train_epochs,
        "learning_rate": training_args.learning_rate,
        "batch_size": training_args.per_device_train_batch_size,
        "effective_batch_size": training_args.per_device_train_batch_size * training_args.gradient_accumulation_steps
    },
    "dataset_info": {
        "total_original_samples": len(combined_clean),
        "sampling_strategy": "random",
        "split_strategy": "80/20 train/test"
    }
}

with open(sample_model_dir / "sample_training_metadata.json", 'w', encoding='utf-8') as f:
    json.dump(metadata, f, indent=2, ensure_ascii=False)

# Calculate model size
model_size_mb = sum(f.stat().st_size for f in sample_model_dir.rglob('*') if f.is_file()) / (1024 * 1024)

print(f"✅ Sample model saved:")
print(f"   📂 Location: {sample_model_dir}")
print(f"   💾 Size: {model_size_mb:.1f} MB")
print(f"   📄 Files: pytorch_model.bin, config.json, tokenizer files, metadata")

💾 Saving sample model...
✅ Sample model saved:
   📂 Location: f:\UOK Fourth Year\Research\my research\dataset\sinhala asr\sample_models\whisper-base-sinhala-sample-final
   💾 Size: 278.9 MB
   📄 Files: pytorch_model.bin, config.json, tokenizer files, metadata
✅ Sample model saved:
   📂 Location: f:\UOK Fourth Year\Research\my research\dataset\sinhala asr\sample_models\whisper-base-sinhala-sample-final
   💾 Size: 278.9 MB
   📄 Files: pytorch_model.bin, config.json, tokenizer files, metadata


## 13. Training Summary & Next Steps

In [30]:
# Final comprehensive summary
print("📊 SAMPLE TRAINING SUMMARY")
print("="*60)
print(f"🎯 Model: Whisper-base fine-tuned on Sinhala sample")
print(f"📊 Sample Data: {len(train_dataset) + len(test_dataset)} total samples")
print(f"   • Training: {len(train_dataset)} samples")
print(f"   • Testing: {len(test_dataset)} samples")
print(f"   • Source: {len(combined_clean):,} original clean samples")
print(f"\n⏱️ Training Time: {train_result.metrics['train_runtime']/60:.1f} minutes")
print(f"🏃 Epochs: {training_args.num_train_epochs}")
print(f"📈 Learning Rate: {training_args.learning_rate}")
print(f"\n📊 Performance Results:")
print(f"   📉 Baseline Loss: {baseline_loss:.4f}")
print(f"   📈 Final Loss: {final_loss:.4f}")
print(f"   📊 Loss Improvement: {improvement:.2f}%")
print(f"   🎯 Test WER: {overall_wer*100:.2f}%")
print(f"   {'🎉' if improvement > 0 else '📊'} Improvement: {improvement:+.2f} percentage points")
print(f"\n💾 Model Info:")
print(f"   📂 Saved to: {sample_model_dir}")
print(f"   💾 Size: {model_size_mb:.1f} MB")
print(f"   🧠 Parameters: {model.num_parameters():,}")

print(f"\n🚀 NEXT STEPS FOR FULL TRAINING:")
print(f"   1. 📈 Use full dataset ({len(combined_clean):,} samples)")
print(f"   2. 🔧 Adjust batch size based on GPU memory")
print(f"   3. ⏰ Plan for longer training time (several hours)")
print(f"   4. 📊 Monitor training with TensorBoard")
print(f"   5. 🎤 Test with real audio files")
print(f"   6. 📝 Consider data augmentation techniques")

print(f"\n💡 CURRENT SAMPLE PERFORMANCE:")
if improvement > 0:
    print(f"   ✅ Model shows improvement on sample data")
    print(f"   ✅ Training pipeline works correctly")
    print(f"   ✅ Ready to scale to full dataset")
else:
    print(f"   ⚠️ Limited improvement on small sample")
    print(f"   ⚠️ Consider: more epochs, different LR, larger sample")
    print(f"   ⚠️ Small sample may not be representative")

print(f"\n✅ Sample training pipeline completed successfully!")
print("="*60)

📊 SAMPLE TRAINING SUMMARY
🎯 Model: Whisper-base fine-tuned on Sinhala sample
📊 Sample Data: 10 total samples
   • Training: 8 samples
   • Testing: 2 samples
   • Source: 90,194 original clean samples

⏱️ Training Time: 1.7 minutes
🏃 Epochs: 3
📈 Learning Rate: 5e-05

📊 Performance Results:
   📉 Baseline Loss: 2.8509
   📈 Final Loss: 2.6045
   📊 Loss Improvement: 0.25%
   🎯 Test WER: 142.86%
   🎉 Improvement: +0.25 percentage points

💾 Model Info:
   📂 Saved to: f:\UOK Fourth Year\Research\my research\dataset\sinhala asr\sample_models\whisper-base-sinhala-sample-final
   💾 Size: 278.9 MB
   🧠 Parameters: 72,593,920

🚀 NEXT STEPS FOR FULL TRAINING:
   1. 📈 Use full dataset (90,194 samples)
   2. 🔧 Adjust batch size based on GPU memory
   3. ⏰ Plan for longer training time (several hours)
   4. 📊 Monitor training with TensorBoard
   5. 🎤 Test with real audio files
   6. 📝 Consider data augmentation techniques

💡 CURRENT SAMPLE PERFORMANCE:
   ✅ Model shows improvement on sample data
   ✅ 

## 14. External Audio Testing with test_audio.wav

Test the trained model with an external audio file to validate real-world performance.

In [46]:
# Load and test external audio file
print("🎤 Testing trained model with external audio file...")

# Path to the test audio file
test_audio_path = data_dir / "test_audio.wav"

try:
    # Check if file exists
    if not test_audio_path.exists():
        print(f"❌ Audio file not found: {test_audio_path}")
        print(f"📂 Looking in directory: {data_dir}")
        print(f"📋 Available files in directory:")
        for file in data_dir.glob("*.wav"):
            print(f"   • {file.name}")
    else:
        print(f"✅ Found audio file: {test_audio_path}")
        
        # Load and validate audio
        audio, sr = librosa.load(test_audio_path, sr=16000)
        duration = len(audio) / sr
        print(f"🎵 Audio Info:")
        print(f"   📊 Duration: {duration:.2f} seconds")
        print(f"   📊 Sample rate: {sr} Hz")
        print(f"   📊 Samples: {len(audio):,}")
        
        if duration < 0.1:
            print("⚠️ Warning: Audio too short (< 0.1s)")
        elif duration > 30:
            print("⚠️ Warning: Audio too long (> 30s), truncating...")
            audio = audio[:30*sr]
            duration = 30.0
        
        # Prepare audio for Whisper
        print(f"\n🔄 Processing audio with trained model...")
        
        # Extract features using the same feature extractor
        input_features = feature_extractor(
            audio, 
            sampling_rate=16000, 
            return_tensors="pt"
        ).input_features
        
        print(f"   📊 Input features shape: {input_features.shape}")
        
        # Generate prediction with the trained model
        with torch.no_grad():
            predicted_ids = model.generate(
                input_features,
                max_length=225,
                num_beams=1,
                do_sample=False,
                language="si",
                task="transcribe"
            )
        
        # Decode the prediction
        prediction = tokenizer.decode(predicted_ids[0], skip_special_tokens=True)
        
        print(f"\n🎯 EXTERNAL AUDIO TEST RESULTS:")
        print("="*60)
        print(f"📁 File: {test_audio_path.name}")
        print(f"⏱️ Duration: {duration:.2f}s")
        print(f"🤖 Prediction: {prediction}")
        print("="*60)
        
        # Additional analysis
        if prediction.strip():
            print(f"\n📊 Prediction Analysis:")
            print(f"   📝 Length: {len(prediction)} characters")
            print(f"   🔤 Words: {len(prediction.split())} words")
            print(f"   🎯 Language detected: Sinhala-aware model")
            
            # Check if it looks like Sinhala romanized text
            sinhala_indicators = ['a', 'i', 'u', 'e', 'o', 'ka', 'ga', 'cha', 'ja', 'ta', 'da', 'na', 'pa', 'ba', 'ma', 'ya', 'ra', 'la', 'va', 'sa', 'ha']
            found_sinhala = any(indicator in prediction.lower() for indicator in sinhala_indicators)
            if found_sinhala:
                print(f"   ✅ Contains Sinhala phonetic patterns")
            else:
                print(f"   ⚠️ May not contain clear Sinhala patterns")
        else:
            print(f"\n⚠️ Empty prediction - model may need more training")
            
except Exception as e:
    print(f"❌ Error processing audio file: {e}")
    print(f"💡 Make sure test_audio.wav is in the correct directory: {data_dir}")

print(f"\n✅ External audio testing completed!")

🎤 Testing trained model with external audio file...
✅ Found audio file: f:\UOK Fourth Year\Research\my research\dataset\sinhala asr\test_audio.wav
🎵 Audio Info:
   📊 Duration: 2.78 seconds
   📊 Sample rate: 16000 Hz
   📊 Samples: 44,446

🔄 Processing audio with trained model...
   📊 Input features shape: torch.Size([1, 80, 3000])

🎯 EXTERNAL AUDIO TEST RESULTS:
📁 File: test_audio.wav
⏱️ Duration: 2.78s
🤖 Prediction:  aam nevotabadigini.

📊 Prediction Analysis:
   📝 Length: 20 characters
   🔤 Words: 2 words
   🎯 Language detected: Sinhala-aware model
   ✅ Contains Sinhala phonetic patterns

✅ External audio testing completed!


In [ ]:
# Improved Romanized to Sinhala conversion with better mapping
def improved_romanized_to_sinhala(romanized_text):
    """Enhanced converter for romanized Sinhala to proper Sinhala Unicode"""
    
    # Comprehensive romanization mapping
    conversion_map = {
        # Complete vowels
        'a': 'අ', 'aa': 'ආ', 'aaa': 'ආ',
        'i': 'ඉ', 'ii': 'ඊ', 'iii': 'ඊ',
        'u': 'උ', 'uu': 'ඌ', 'uuu': 'ඌ',
        'e': 'එ', 'ee': 'ඒ', 'eee': 'ඒ',
        'o': 'ඔ', 'oo': 'ඕ', 'ooo': 'ඕ',
        'au': 'ඖ', 'ow': 'ඖ',
        
        # Consonants with vowel 'a'
        'ka': 'ක', 'kha': 'ඛ', 'ga': 'ග', 'gha': 'ඝ', 'nga': 'ඞ',
        'cha': 'ච', 'chha': 'ඡ', 'ja': 'ජ', 'jha': 'ඣ', 'nya': 'ඤ',
        'tta': 'ට', 'ttha': 'ඨ', 'dda': 'ඩ', 'ddha': 'ඪ', 'nna': 'ණ',
        'ta': 'ත', 'tha': 'ථ', 'da': 'ද', 'dha': 'ධ', 'na': 'න',
        'pa': 'ප', 'pha': 'ෆ', 'ba': 'බ', 'bha': 'භ', 'ma': 'ම',
        'ya': 'ය', 'ra': 'ර', 'la': 'ල', 'va': 'ව', 'wa': 'ව',
        'sha': 'ශ', 'sa': 'ස', 'ha': 'හ', 'lla': 'ළ', 'fa': 'ෆ',
        
        # Consonants without vowel
        'k': 'ක්', 'g': 'ග්', 'ng': 'ං', 'ch': 'ච්', 'j': 'ජ්',
        't': 'ත්', 'd': 'ද්', 'n': 'න්', 'p': 'ප්', 'b': 'බ්',
        'm': 'ම්', 'y': 'ය්', 'r': 'ර්', 'l': 'ල්', 'v': 'ව්',
        'w': 'ව්', 's': 'ස්', 'h': 'හ්', 'f': 'ෆ්',
        
        # Common combinations and words
        'me': 'මේ', 'mee': 'මී', 'mama': 'මම', 'maama': 'මාමා',
        'api': 'අපි', 'eka': 'එක', 'eke': 'එකේ',
        'katha': 'කතා', 'kathaa': 'කතා', 
        'hoda': 'හොඳ', 'hodai': 'හොඳයි', 'honda': 'හොඳ',
        'ane': 'අනේ', 'aam': 'ආම්', 'ama': 'අම',
        'ne': 'නේ', 'nae': 'නෑ', 'neh': 'නෙහ්',
        'muta': 'මුට', 'mutaa': 'මුටා', 'mata': 'මට',
        'badi': 'බදි', 'baddi': 'බද්දි',
        'gini': 'ගිනි', 'gini': 'ගිණි',
        'digi': 'දිගි', 'dini': 'දිනි',
        'mutaba': 'මුතබ', 'badigini': 'බදිගිනි',
        
        # Special characters and punctuation
        '.': '।', ',': '،', '!': '!', '?': '?'
    }
    
    # Clean the text
    text = romanized_text.strip().lower()
    
    # Split into words and process each
    words = text.split()
    sinhala_words = []
    
    for word in words:
        # Remove punctuation for processing but remember it
        punctuation = ''
        clean_word = word
        if word and word[-1] in '.!?,:;':
            punctuation = word[-1]
            clean_word = word[:-1]
        
        # Convert the word
        converted_word = convert_word_improved(clean_word, conversion_map)
        
        # Add punctuation back
        if punctuation:
            if punctuation in conversion_map:
                converted_word += conversion_map[punctuation]
            else:
                converted_word += punctuation
        
        sinhala_words.append(converted_word)
    
    return ' '.join(sinhala_words)

def convert_word_improved(word, conversion_map):
    """Convert a single word using the conversion map with better logic"""
    if not word:
        return word
    
    # Try direct mapping first
    if word in conversion_map:
        return conversion_map[word]
    
    # Try to break down the word
    result = ""
    remaining = word
    
    # Sort conversion keys by length (longest first) for better matching
    sorted_keys = sorted(conversion_map.keys(), key=len, reverse=True)
    
    while remaining:
        found = False
        for pattern in sorted_keys:
            if remaining.startswith(pattern):
                result += conversion_map[pattern]
                remaining = remaining[len(pattern):]
                found = True
                break
        
        if not found:
            # If no pattern matches, keep the character as is
            result += remaining[0]
            remaining = remaining[1:]
    
    return result

# Test the improved conversion
test_text = "aam ne mutabadigini"
improved_result = improved_romanized_to_sinhala(test_text)

print(f"🔤 Improved Sinhala Conversion Test:")
print(f"   📝 Input: '{test_text}'")
print(f"   🇱🇰 Output: '{improved_result}'")
print(f"✅ Improved conversion function ready!")